In [17]:
import os
import pandas as pd
import numpy as np
from collections import Counter
os.chdir("UCI HAR Dataset/UCI HAR Dataset/test")
x_test = pd.read_csv("X_test.txt", "\s+", header=None)
y_test = pd.read_csv("y_test.txt", "\s+", header=None)
os.chdir("UCI HAR Dataset/UCI HAR Dataset/train")
x_train = pd.read_csv("X_train.txt", "\s+", header=None)
y_train = pd.read_csv("y_train.txt", "\s+", header=None)

In [99]:
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

(2947, 561)
(2947, 1)
(7352, 561)
(7352, 1)


### Normalize values

In [18]:
xmin = x_train.min()
xmax = x_train.max()
x_train = x_train-xmin/(xmax-xmin)
x_test = x_test-xmin/(xmax-xmin)

### View individuals

In [19]:
os.chdir("UCI HAR Dataset/UCI HAR Dataset/test")
text = pd.read_csv("subject_test.txt")
# text.iloc[617]
# text[text["2"]==4]
text.iloc[0:301]["2"].unique()
text.iloc[618:906]["2"].unique()
text.iloc[301:618]["2"].unique()
text.iloc[906:1200]["2"].unique()
text.iloc[1200:1520]["2"].unique()
text.iloc[1520:1847]["2"].unique()
text.iloc[1847:2211]["2"].unique()
text.iloc[2211:2565]["2"].unique()
text.iloc[2565:2946]["2"].unique()

array([24], dtype=int64)

### Check Class Balance

In [102]:
print(len(y_train[y_train[0]==1]))
print(len(y_train[y_train[0]==2]))
print(len(y_train[y_train[0]==3]))
print(len(y_train[y_train[0]==4]))
print(len(y_train[y_train[0]==5]))
print(len(y_train[y_train[0]==6]))
print(len(y_test[y_test[0]==1]))
print(len(y_test[y_test[0]==2]))
print(len(y_test[y_test[0]==3]))
print(len(y_test[y_test[0]==4]))
print(len(y_test[y_test[0]==5]))
print(len(y_test[y_test[0]==6]))

1226
1073
986
1286
1374
1407
496
471
420
491
532
537


### Create Validation Set

In [20]:
x_valid = x_test.iloc[0:1520]
x_test = x_test.iloc[1520:2946]
y_valid = y_test.iloc[0:1520]
y_test = y_test.iloc[1520:2946]

### Iteratively drop columns XGBoost

In [67]:
# # First Iteration
# select_columns = np.r_[0:561]
# # Second Iteration
# select_columns = np.r_[0:19,20:28,29:39,40,42:50,55:60,61:94,95:108,113,114,120,123,124,126,131:137,140,141,143:150,151:154,156,157,159:163,165:168,170,172:177,182:186,187:195,196:204,210,213,215:221,224,245,246,248,249,255,263,271,282,300,301,318,319,321,322,325,330,331,333,342,344,347,348,350,354:357,369,372:375,388:392,398:406,407,408,410,413,417:422,426,427,429:444,447,448,450,452:458,459:467,468:473,474:513,514:527,529:533,534,535,538:561]
# # Third Iteration
select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # Fourth Iteration
select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
# # Fifth Iteration
select_columns = select_columns[np.r_[0,1,3:10,11:14,15,16,18,19,23,24,26,28:31,32,34:46,48,51,53:59,62:66,67,69:72,75:78,82,86,107]]
x_train_temp = x_train.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
import xgboost
gridXG = ParameterGrid({"n_estimators": [10],
                        "learning_rate": [.8],
                        "max_depth": [4],
                        "booster": ["dart"], #"gbtree", "gblinear"
                        "subsample": [.6],
                        "colsample_bytree": [.8],
                        "colsample_bylevel": [1],
                        "n_jobs": [4],
                        "gamma": [0],
                        "min_child_weight": [.4],
                        "max_delta_step": [4],
                        "colsample_bynode": [.9],
                        "reg_alpha": [.1],
                        "random_state": [5,15,35,55,71,88,104,112,128,161]
                       })
nums = []
for i in range(0,561):
    print(i)
    x_train_sub = x_train_temp.drop(x_train_temp.columns[i], axis=1)
    x_valid_sub = x_valid_temp.drop(x_valid_temp.columns[i], axis=1)
    for j, params in enumerate(gridXG):
        start = time.time()
        model = xgboost.XGBClassifier(**params)
        model.fit(x_train_sub, y_train)
        score = model.predict(x_valid_sub)
        cm = confusion_matrix(y_valid, score)
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("accuracy_full: {}".format(sum(cm.diagonal())/6))
        precision, recall, fscore, support = precision_recall_fscore_support(y_valid, score)
        print('precision_full: {}'.format(precision))
        print('recall_full: {}'.format(recall))
        end = time.time()
        print(end-start)
        if sum(cm.diagonal())/6<.935:
            nums.append(str(i)+" - "+str(j))


0


C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy_full: 0.9382537711786788
precision_full: [0.91304348 0.98636364 0.95022624 0.92307692 0.87681159 1.        ]
recall_full: [0.99635036 0.87854251 0.96330275 0.86055777 0.93076923 1.        ]
1.4232831001281738
accuracy_full: 0.9406903056609882
precision_full: [0.95759717 0.97826087 0.9380531  0.91416309 0.86330935 1.        ]
recall_full: [0.98905109 0.91093117 0.97247706 0.84860558 0.92307692 1.        ]
1.7290890216827393
accuracy_full: 0.9385458021781071
precision_full: [0.92542373 0.96902655 0.94954128 0.93859649 0.86925795 1.        ]
recall_full: [0.99635036 0.88663968 0.94954128 0.85258964 0.94615385 1.        ]
1.7618274688720703
accuracy_full: 0.9380426864359476
precision_full: [0.93493151 0.96902655 0.94090909 0.91286307 0.88191882 1.        ]
recall_full: [0.99635036 0.88663968 0.94954128 0.87649402 0.91923077 1.        ]
1.466092586517334
accuracy_full: 0.9349750959385502
precision_full: [0.93684211 0.97402597 0.9321267  0.91845494 0.86071429 1.        ]
recall_full

accuracy_full: 0.9335505797030538
precision_full: [0.94097222 0.9650655  0.93243243 0.90295359 0.86861314 1.        ]
recall_full: [0.98905109 0.89473684 0.94954128 0.85258964 0.91538462 1.        ]
2.8146615028381348
accuracy_full: 0.9368164472439479
precision_full: [0.95698925 0.96995708 0.9254386  0.91703057 0.85765125 1.        ]
recall_full: [0.97445255 0.91497976 0.96788991 0.83665339 0.92692308 1.        ]
2.516819715499878
4
accuracy_full: 0.924628668516565
precision_full: [0.90033223 0.98148148 0.94196429 0.89270386 0.85144928 1.        ]
recall_full: [0.98905109 0.8582996  0.96788991 0.82868526 0.90384615 1.        ]
2.5146987438201904
accuracy_full: 0.9318305837885816
precision_full: [0.91610738 0.98173516 0.94196429 0.89873418 0.86764706 1.        ]
recall_full: [0.99635036 0.87044534 0.96788991 0.84860558 0.90769231 1.        ]
2.428497791290283
accuracy_full: 0.9352457315239538
precision_full: [0.93150685 0.98654709 0.94222222 0.91629956 0.85159011 1.        ]
recall_full

accuracy_full: 0.9403115150167825
precision_full: [0.93793103 0.96536797 0.95       0.91949153 0.87912088 1.        ]
recall_full: [0.99270073 0.90283401 0.9587156  0.86454183 0.92307692 1.        ]
2.2495808601379395
accuracy_full: 0.9379763226115467
precision_full: [0.92517007 0.96982759 0.95327103 0.89558233 0.89655172 1.        ]
recall_full: [0.99270073 0.91093117 0.93577982 0.88844622 0.9        1.        ]
1.9987154006958008
accuracy_full: 0.9311415398743543
precision_full: [0.91467577 0.97272727 0.91666667 0.91525424 0.87912088 1.        ]
recall_full: [0.97810219 0.86639676 0.9587156  0.86055777 0.92307692 1.        ]
1.9451508522033691
accuracy_full: 0.946611934068626
precision_full: [0.95470383 0.98245614 0.94666667 0.91286307 0.89179104 0.99630996]
recall_full: [1.         0.90688259 0.97706422 0.87649402 0.91923077 1.        ]
2.0716159343719482
8
accuracy_full: 0.9247256550509344
precision_full: [0.92439863 0.98214286 0.93333333 0.89035088 0.83333333 1.        ]
recall_fu

...

accuracy_full: 0.924308824202602
precision_full: [0.89508197 0.95555556 0.96190476 0.89495798 0.86397059 1.        ]
recall_full: [0.99635036 0.87044534 0.9266055  0.84860558 0.90384615 1.        ]
2.110860824584961
accuracy_full: 0.9457410625206855
precision_full: [0.96402878 0.98283262 0.92608696 0.90243902 0.8973384  1.        ]
recall_full: [0.97810219 0.92712551 0.97706422 0.88446215 0.90769231 1.        ]
2.1349966526031494
42
accuracy_full: 0.9358125972644243
precision_full: [0.92176871 0.98222222 0.94570136 0.89711934 0.88014981 1.        ]
recall_full: [0.98905109 0.89473684 0.9587156  0.8685259  0.90384615 1.        ]
2.969480514526367
accuracy_full: 0.9461772388109053
precision_full: [0.95422535 0.9790795  0.95852535 0.94570136 0.85813149 1.        ]
recall_full: [0.98905109 0.94736842 0.95412844 0.83266932 0.95384615 1.        ]
2.1605050563812256
accuracy_full: 0.9433431051738288
precision_full: [0.96774194 0.97468354 0.94196429 0.91774892 0.86379928 1.        ]
recall_ful

accuracy_full: 0.9251622775042537
precision_full: [0.94791667 0.98689956 0.94594595 0.89671362 0.79865772 1.        ]
recall_full: [0.99635036 0.91497976 0.96330275 0.76095618 0.91538462 1.        ]
2.035447835922241
accuracy_full: 0.9285517790840139
precision_full: [0.93793103 0.97379913 0.94594595 0.89380531 0.83392226 1.        ]
recall_full: [0.99270073 0.90283401 0.96330275 0.80478088 0.90769231 1.        ]
1.9472808837890625
accuracy_full: 0.913468675036856
precision_full: [0.91610738 0.97727273 0.9321267  0.87782805 0.80344828 1.        ]
recall_full: [0.99635036 0.87044534 0.94495413 0.77290837 0.89615385 1.        ]
1.938577651977539
accuracy_full: 0.9183490547142493
precision_full: [0.92517007 0.96875    0.93693694 0.88687783 0.81597222 0.99630996]
recall_full: [0.99270073 0.87854251 0.95412844 0.78087649 0.90384615 1.        ]
2.344001054763794
46
accuracy_full: 0.9381075736564458
precision_full: [0.94097222 0.98268398 0.95022624 0.87649402 0.88416988 1.        ]
recall_full

accuracy_full: 0.9378710876540906
precision_full: [0.9209622  0.97413793 0.93518519 0.91735537 0.89219331 1.        ]
recall_full: [0.97810219 0.91497976 0.9266055  0.88446215 0.92307692 1.        ]
2.0212454795837402
accuracy_full: 0.9389402404670965
precision_full: [0.94444444 0.96137339 0.92694064 0.93506494 0.8781362  1.        ]
recall_full: [0.99270073 0.90688259 0.93119266 0.86055777 0.94230769 1.        ]
2.019463300704956
accuracy_full: 0.9397286911167012
precision_full: [0.93793103 0.98230088 0.93721973 0.90204082 0.88721805 1.        ]
recall_full: [0.99270073 0.89878543 0.9587156  0.88047809 0.90769231 1.        ]
2.0825462341308594
accuracy_full: 0.9358873781766316
precision_full: [0.94385965 0.96551724 0.93721973 0.90677966 0.86861314 1.        ]
recall_full: [0.98175182 0.90688259 0.9587156  0.85258964 0.91538462 1.        ]
2.262047290802002
accuracy_full: 0.9316942799580578
precision_full: [0.93771626 0.96069869 0.93273543 0.89873418 0.86764706 1.        ]
recall_full:

accuracy_full: 0.9289345592636217
precision_full: [0.93425606 0.97321429 0.92035398 0.92376682 0.84027778 1.        ]
recall_full: [0.98540146 0.88259109 0.95412844 0.82071713 0.93076923 1.        ]
1.9845857620239258
accuracy_full: 0.9151929009541218
precision_full: [0.89108911 0.97663551 0.91441441 0.89565217 0.83985765 1.        ]
recall_full: [0.98540146 0.84615385 0.93119266 0.82071713 0.90769231 1.        ]
1.9927940368652344
accuracy_full: 0.9405347185689527
precision_full: [0.9375     0.96666667 0.95305164 0.92672414 0.87364621 1.        ]
recall_full: [0.98540146 0.93927126 0.93119266 0.85657371 0.93076923 1.        ]
2.0412206649780273
accuracy_full: 0.9244824362226479
precision_full: [0.9375     0.95964126 0.89082969 0.92410714 0.84965035 1.        ]
recall_full: [0.98540146 0.86639676 0.93577982 0.8247012  0.93461538 1.        ]
2.177722930908203
accuracy_full: 0.9321682989027936
precision_full: [0.9220339  0.98206278 0.93243243 0.90295359 0.86813187 1.        ]
recall_full

IndexError: index 57 is out of bounds for axis 0 with size 57

### Test results and parameters on validation XGBoost

In [68]:
# # First Iteration
# select_columns = np.r_[0:561]
# # Second Iteration
# select_columns = np.r_[0:19,20:28,29:39,40,42:50,55:60,61:94,95:108,113,114,120,123,124,126,131:137,140,141,143:150,151:154,156,157,159:163,165:168,170,172:177,182:186,187:195,196:204,210,213,215:221,224,245,246,248,249,255,263,271,282,300,301,318,319,321,322,325,330,331,333,342,344,347,348,350,354:357,369,372:375,388:392,398:406,407,408,410,413,417:422,426,427,429:444,447,448,450,452:458,459:467,468:473,474:513,514:527,529:533,534,535,538:561]
# # Third Iteration
select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # Fourth Iteration
select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
# # Fifth Iteration
# select_columns = select_columns[np.r_[0,1,3:10,11:14,15,16,18,19,23,24,26,28:31,32,34:46,48,51,53:59,62:66,67,69:72,75:78,82,86,107]]
# # # Sixth Iteration
# select_columns = select_columns[np.r_[3,4,6,8:14,18:21,23:26,28,30,34,36,40,41,45,46,48:51,53,54,56]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
import xgboost
gridXG = ParameterGrid({"n_estimators": [120],
                        "learning_rate": [.1],
                        "max_depth": [5],
                        "booster": ["dart"], #"gbtree", "gblinear"
                        "subsample": [.2],
                        "colsample_bytree": [1],
                        "colsample_bylevel": [1],
                        "n_jobs": [4],
                        "gamma": [0],
                        "min_child_weight": [.1],
                        "max_delta_step": [4],
                        "colsample_bynode": [1],
                        "reg_alpha": [0], 
                        "random_state": [1]
                       })

for i, params in enumerate(gridXG):
    start = time.time()
    model = xgboost.XGBClassifier(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_valid_temp)
    cm = confusion_matrix(y_valid, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    precision, recall, fscore, support = precision_recall_fscore_support(y_valid, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(i, params)
    print(end-start)
    

accuracy_full: 0.9681184312015887
precision_full: [0.97857143 0.99585062 0.98173516 0.93469388 0.92075472 1.        ]
recall_full: [1.         0.97165992 0.98623853 0.9123506  0.93846154 1.        ]
0 {'booster': 'dart', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 4, 'max_depth': 5, 'min_child_weight': 0.1, 'n_estimators': 120, 'n_jobs': 4, 'random_state': 1, 'reg_alpha': 0, 'subsample': 0.2}
34.20910310745239


In [71]:
iteration = []
seed = []
for i in nums:
    iteration.append(int(i.split(" - ")[0]))
    seed.append(int(i.split(" - ")[1]))

In [49]:
# Counter(iteration)

### Final Test XGBoost

In [72]:
select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # Fourth Iteration
select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
import xgboost
gridXG = ParameterGrid({"n_estimators": [120],
                        "learning_rate": [.1],
                        "max_depth": [5],
                        "booster": ["dart"], #"gbtree", "gblinear"
                        "subsample": [.2],
                        "colsample_bytree": [1],
                        "colsample_bylevel": [1],
                        "n_jobs": [4],
                        "gamma": [0],
                        "min_child_weight": [.1],
                        "max_delta_step": [4],
                        "colsample_bynode": [1],
                        "reg_alpha": [0], 
                        "random_state": [210,220,230,240,250,260,270,280,290,300]
                       })

for params in gridXG:
    start = time.time()
    model = xgboost.XGBClassifier(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_test_temp)
    cm = confusion_matrix(y_test, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    print("accuracy_full: {}".format(cm.diagonal()))
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(end-start)

accuracy_full: 0.9449378105537104
accuracy_full: [0.95495495 1.         0.84653465 0.87916667 0.98897059 1.        ]
precision_full: [0.96803653 0.88844622 0.96610169 0.98598131 0.90268456 1.        ]
recall_full: [0.95495495 1.         0.84653465 0.87916667 0.98897059 1.        ]
40.81721758842468
accuracy_full: 0.9516543437828359
accuracy_full: [0.97297297 1.         0.87623762 0.87916667 0.98529412 0.99625468]
precision_full: [0.96860987 0.91393443 0.98333333 0.97685185 0.9023569  1.        ]
recall_full: [0.97297297 1.         0.87623762 0.87916667 0.98529412 0.99625468]
41.113261461257935
accuracy_full: 0.9447342539201135
accuracy_full: [0.97747748 1.         0.83168317 0.86666667 0.99632353 0.99625468]
precision_full: [0.96875    0.88492063 0.98245614 0.99047619 0.89438944 1.        ]
recall_full: [0.97747748 1.         0.83168317 0.86666667 0.99632353 0.99625468]
40.79485726356506
accuracy_full: 0.9503482801478554
accuracy_full: [0.96846847 1.         0.86138614 0.88333333 0.992

### Test results and parameters on validation Adaboost

In [74]:
# # First Iteration
# select_columns = np.r_[0:561]
# # Second Iteration
# select_columns = np.r_[0:19,20:28,29:39,40,42:50,55:60,61:94,95:108,113,114,120,123,124,126,131:137,140,141,143:150,151:154,156,157,159:163,165:168,170,172:177,182:186,187:195,196:204,210,213,215:221,224,245,246,248,249,255,263,271,282,300,301,318,319,321,322,325,330,331,333,342,344,347,348,350,354:357,369,372:375,388:392,398:406,407,408,410,413,417:422,426,427,429:444,447,448,450,452:458,459:467,468:473,474:513,514:527,529:533,534,535,538:561]
# # Third Iteration
select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # Fourth Iteration
select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
# # Fifth Iteration
# select_columns = select_columns[np.r_[0,1,3:10,11:14,15,16,18,19,23,24,26,28:31,32,34:46,48,51,53:59,62:66,67,69:72,75:78,82,86,107]]
# # # Sixth Iteration
# select_columns = select_columns[np.r_[3,4,6,8:14,18:21,23:26,28,30,34,36,40,41,45,46,48:51,53,54,56]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.ensemble import AdaBoostClassifier
gridAB = ParameterGrid({"n_estimators": [160],
                        "learning_rate": [.8],
                        })

for i, params in enumerate(gridAB):
    start = time.time()
    model = AdaBoostClassifier(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_valid_temp)
    cm = confusion_matrix(y_valid, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    precision, recall, fscore, support = precision_recall_fscore_support(y_valid, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(i, params)
    print(end-start)
    

C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy_full: 0.691961569679297
precision_full: [0.93714286 0.85820896 0.49767442 0.62886598 0.55188679 1.        ]
recall_full: [0.59854015 0.46558704 0.98165138 0.24302789 0.9        0.96296296]
0 {'learning_rate': 0.8, 'n_estimators': 160}
20.19591760635376


### Final Test Adaboost

In [75]:
select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # Fourth Iteration
select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.ensemble import AdaBoostClassifier
gridAB = ParameterGrid({"n_estimators": [160],
                        "learning_rate": [.8]
                        })

for params in gridAB:
    start = time.time()
    model = AdaBoostClassifier(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_test_temp)
    cm = confusion_matrix(y_test, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    print("accuracy_full: {}".format(cm.diagonal()))
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(end-start)

C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy_full: 0.7083252242504541
accuracy_full: [0.54954955 0.69058296 0.83663366 0.225      0.96691176 0.98127341]
precision_full: [0.89051095 0.79381443 0.53481013 0.79411765 0.5857461  1.        ]
recall_full: [0.54954955 0.69058296 0.83663366 0.225      0.96691176 0.98127341]
21.15660047531128


### Test results and parameters on validation Random Forest

In [21]:
# # First Iteration
# select_columns = np.r_[0:561]
# # Second Iteration
# select_columns = np.r_[0:19,20:28,29:39,40,42:50,55:60,61:94,95:108,113,114,120,123,124,126,131:137,140,141,143:150,151:154,156,157,159:163,165:168,170,172:177,182:186,187:195,196:204,210,213,215:221,224,245,246,248,249,255,263,271,282,300,301,318,319,321,322,325,330,331,333,342,344,347,348,350,354:357,369,372:375,388:392,398:406,407,408,410,413,417:422,426,427,429:444,447,448,450,452:458,459:467,468:473,474:513,514:527,529:533,534,535,538:561]
# # Third Iteration
select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # Fourth Iteration
select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
# # Fifth Iteration
# select_columns = select_columns[np.r_[0,1,3:10,11:14,15,16,18,19,23,24,26,28:31,32,34:46,48,51,53:59,62:66,67,69:72,75:78,82,86,107]]
# # # Sixth Iteration
# select_columns = select_columns[np.r_[3,4,6,8:14,18:21,23:26,28,30,34,36,40,41,45,46,48:51,53,54,56]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
gridRF = ParameterGrid({"n_estimators": [120],
                        "min_samples_split": [.01],
                        "max_features": [70],
                        "max_depth": [9],
                        "min_samples_leaf": [13],
                        "bootstrap": ["True"],
                        "max_leaf_nodes": [50],
                        "n_jobs": [4],
                        })

for i, params in enumerate(gridRF):
    start = time.time()
    model = RandomForestClassifier(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_valid_temp)
    cm = confusion_matrix(y_valid, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    precision, recall, fscore, support = precision_recall_fscore_support(y_valid, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(i, params)
    print(end-start)
    

C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


accuracy_full: 0.9252209747169396
precision_full: [0.88778878 0.99078341 0.95454545 0.91479821 0.83623693 1.        ]
recall_full: [0.98175182 0.87044534 0.96330275 0.812749   0.92307692 1.        ]
0 {'bootstrap': 'True', 'max_depth': 9, 'max_features': 70, 'max_leaf_nodes': 50, 'min_samples_leaf': 13, 'min_samples_split': 0.01, 'n_estimators': 120, 'n_jobs': 4}
13.751423597335815


### Final Test Random Forest

In [112]:
select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # Fourth Iteration
select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
gridRF = ParameterGrid({"n_estimators": [120],
                        "min_samples_split": [.01],
                        "max_features": [70],
                        "max_depth": [9],
                        "min_samples_leaf": [13],
                        "bootstrap": ["True"],
                        "max_leaf_nodes": [50],
                        "n_jobs": [4],
                        })

for params in gridRF:
    start = time.time()
    model = RandomForestClassifier(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_test_temp)
    cm = confusion_matrix(y_test, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    print("accuracy_full: {}".format(cm.diagonal()))
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(end-start)

C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


accuracy_full: 0.9027000918580748
accuracy_full: [0.92792793 0.96860987 0.73762376 0.80416667 0.98161765 0.99625468]
precision_full: [0.9321267  0.79120879 0.97385621 0.96984925 0.85031847 1.        ]
recall_full: [0.92792793 0.96860987 0.73762376 0.80416667 0.98161765 0.99625468]
23.135361194610596


### Test results and parameters on validation SVM

In [4]:
#I trained on a different file with train and validation combined.  Results in the final SVM cell appear a little inflated.

# # First Iteration
select_columns = np.r_[0:561]
# # Second Iteration
# select_columns = np.r_[0:19,20:28,29:39,40,42:50,55:60,61:94,95:108,113,114,120,123,124,126,131:137,140,141,143:150,151:154,156,157,159:163,165:168,170,172:177,182:186,187:195,196:204,210,213,215:221,224,245,246,248,249,255,263,271,282,300,301,318,319,321,322,325,330,331,333,342,344,347,348,350,354:357,369,372:375,388:392,398:406,407,408,410,413,417:422,426,427,429:444,447,448,450,452:458,459:467,468:473,474:513,514:527,529:533,534,535,538:561]
# # Third Iteration
# select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # # Fourth Iteration
# select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
# # Fifth Iteration
# select_columns = select_columns[np.r_[0,1,3:10,11:14,15,16,18,19,23,24,26,28:31,32,34:46,48,51,53:59,62:66,67,69:72,75:78,82,86,107]]
# # # Sixth Iteration
# select_columns = select_columns[np.r_[3,4,6,8:14,18:21,23:26,28,30,34,36,40,41,45,46,48:51,53,54,56]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.svm import SVC  
gridSVC = ParameterGrid({"kernel": ["linear"], #, "poly", "rbf", "sigmoid"
                         "gamma": [.1],
                         "degree": [6],
                         "C": [3.5],
#                          "tol": [1],
#                          "random_state": [1],
                         "max_iter": [1000000]
                       })

for i, params in enumerate(gridSVC):
    start = time.time()
    model = SVC(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_valid_temp)
    cm = confusion_matrix(y_valid, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    precision, recall, fscore, support = precision_recall_fscore_support(y_valid, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(i, params)
    print(end-start)
    

C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy_full: 0.9404100047806412
precision_full: [0.9375     0.98275862 0.97747748 0.9468599  0.82724252 1.        ]
recall_full: [0.98540146 0.92307692 0.99541284 0.78087649 0.95769231 1.        ]
0 {'C': 3.5, 'degree': 6, 'gamma': 0.1, 'kernel': 'linear', 'max_iter': 1000000}
3.3136229515075684


### Final Test SVM

In [5]:
select_columns = np.r_[0:561]
# select_columns = np.r_[0,1,4,5,9,11,21,23,38,42,43,45,46,47,48,49,55,56,58,59,61,62,64,65,67,68,69,71,74,75,76,80,81,83,84,89,90,92,93,95,99,100,101,102,103,114,120,126,131,132,134,145,151,152,153,157,176,183,187,188,189,193,194,197,199,202,203,210,213,215,216,218,245,246,248,249,255,263,271,319,330,333,342,350,355,356,388,407,408,410,429,430,432,433,434,435,437,438,439,440,441,442,443,447,448,450,452,453,454,455,456,457,459,460,461,462,463,464,465,466,468,469,472,474,479,486,487,488,491,496,498,499,503,504,506,507,508,509,511,512,514,515,516,517,518,519,520,521,523,524,525,538,539,540,541,542,543,544,548,555,557,558]
# # # Fourth Iteration
# select_columns = select_columns[np.r_[10,11,13,14,17:22,23:28,29:35,36,37,39:47,49,50,52:56,59:65,67:74,75,79,91:94,95,99,104:109,110:120,121:137,138,140:147,148:154,155:162]]
x_train_temp = x_train.iloc[:,select_columns]
x_valid_temp = x_valid.iloc[:,select_columns]
x_test_temp = x_test.iloc[:,select_columns]
from sklearn.model_selection import ParameterGrid
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.svm import SVC  
gridSVC = ParameterGrid({"kernel": ["linear"], #, "poly", "rbf", "sigmoid"
                         "gamma": [.1],
                         "degree": [6],
                         "C": [3.5],
#                          "tol": [1],
#                          "random_state": [1],
                         "max_iter": [1000000]
                       })

for params in gridSVC:
    start = time.time()
    model = SVC(**params)
    model.fit(x_train_temp, y_train)
    score = model.predict(x_test_temp)
    cm = confusion_matrix(y_test, score)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("accuracy_full: {}".format(sum(cm.diagonal())/6))
    print("accuracy_full: {}".format(cm.diagonal()))
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, score)
    print('precision_full: {}'.format(precision))
    print('recall_full: {}'.format(recall))
    end = time.time()
    print(end-start)

C:\Users\jerem.DESKTOP-GGM6Q2I\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy_full: 0.9878236052964304
accuracy_full: [1.         0.9955157  0.95544554 0.98333333 0.99264706 1.        ]
precision_full: [0.98230088 0.97368421 1.         0.99159664 0.98540146 1.        ]
recall_full: [1.         0.9955157  0.95544554 0.98333333 0.99264706 1.        ]
3.3277037143707275


### Read in files

In [13]:
import os
import pandas as pd
import numpy as np
from collections import Counter
os.chdir("UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals")
test1 = pd.read_csv("body_acc_x_test.txt", "\s+", header=None)
test2 = pd.read_csv("body_acc_y_test.txt", "\s+", header=None)
test3 = pd.read_csv("body_acc_z_test.txt", "\s+", header=None)
test4 = pd.read_csv("body_gyro_x_test.txt", "\s+", header=None)
test5 = pd.read_csv("body_gyro_y_test.txt", "\s+", header=None)
test6 = pd.read_csv("body_gyro_z_test.txt", "\s+", header=None)
test7 = pd.read_csv("total_acc_x_test.txt", "\s+", header=None)
test8 = pd.read_csv("total_acc_y_test.txt", "\s+", header=None)
test9 = pd.read_csv("total_acc_z_test.txt", "\s+", header=None)
os.chdir("UCI HAR Dataset/UCI HAR Dataset/train/Inertial Signals")
train1 = pd.read_csv("body_acc_x_train.txt", "\s+", header=None)
train2 = pd.read_csv("body_acc_y_train.txt", "\s+", header=None)
train3 = pd.read_csv("body_acc_z_train.txt", "\s+", header=None)
train4 = pd.read_csv("body_gyro_x_train.txt", "\s+", header=None)
train5 = pd.read_csv("body_gyro_y_train.txt", "\s+", header=None)
train6 = pd.read_csv("body_gyro_z_train.txt", "\s+", header=None)
train7 = pd.read_csv("total_acc_x_train.txt", "\s+", header=None)
train8 = pd.read_csv("total_acc_y_train.txt", "\s+", header=None)
train9 = pd.read_csv("total_acc_z_train.txt", "\s+", header=None)

### Create arrays in correct shape

In [14]:
x_test = np.dstack((test1,test2,test3,test4,test5,test6,test7,test8,test9))
x_train = np.dstack((train1,train2,train3,train4,train5,train6,train7,train8,train9))
os.chdir("UCI HAR Dataset/UCI HAR Dataset/test")
y_test = pd.read_csv("y_test.txt", "\s+", header=None)
os.chdir("UCI HAR Dataset/UCI HAR Dataset/train")
y_train = pd.read_csv("y_train.txt", "\s+", header=None)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(7352, 128, 9)
(2947, 128, 9)
(7352, 1)
(2947, 1)


In [15]:
y_train = pd.get_dummies(y_train[0]).values
y_test = pd.get_dummies(y_test[0]).values
x_train = x_train.reshape([7352,1152])
x_test = x_test.reshape([2947,1152])
print(y_test.shape)
y_test

(2947, 6)


array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [83]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot



Using TensorFlow backend.


### Run ANN

In [84]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam, RMSprop
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
set_random_seed(1724)


model = models.Sequential()
model.add(Dense(128, kernel_initializer="glorot_normal", input_shape=(9*128,)))
model.add(LeakyReLU())
model.add(Dropout(.15))
model.add(BatchNormalization())
# model.add(GaussianNoise(1))
model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dense(6, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [85]:
# Let's use a different optimizer this time
noisy = add_gradient_noise(RMSprop)
model.compile(optimizer="Adamax",
# model.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [86]:
callbacks = [EarlyStopping(monitor='val_loss', patience=16),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [87]:
history = model.fit(x_train, y_train, epochs=250, batch_size=16, validation_data = (x_test, y_test), callbacks=callbacks)
from keras.models import load_model
# model = load_model('best_model.h5', custom_objects={"NoisyRMSprop":noisy()})
model = load_model('best_model.h5')
model.evaluate(x_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/250
7352/7352 [==============================] - 4s 536us/step - loss: 0.8548 - acc: 0.6813 - val_loss: 0.6692 - val_acc: 0.7781
Epoch 2/250
7352/7352 [==============================] - 4s 605us/step - loss: 0.5102 - acc: 0.8243 - val_loss: 0.5082 - val_acc: 0.8344
Epoch 3/250
7352/7352 [==============================] - 5s 713us/step - loss: 0.4016 - acc: 0.8604 - val_loss: 0.4620 - val_acc: 0.8466
Epoch 4/250
7352/7352 [==============================] - 5s 618us/step - loss: 0.3395 - acc: 0.8863 - val_loss: 0.3863 - val_acc: 0.8724
Epoch 5/250
7352/7352 [==============================] - 4s 576us/step - loss: 0.3036 - acc: 0.8934 - val_loss: 0.4075 - val_acc: 0.8419
Epoch 6/250
7352/7352 [==============================] - 4s 604us/step - loss: 0.2754 - acc: 0.9030 - val_loss: 0.3268 - val_acc: 0.8935
Epoch 7/250
7352/7352 [==============================] - 5s 673us/step - loss: 0.2

[0.23868875675294315, 0.9056667797760435]

### Try ANN again with first 64 of 128 time values per row

In [88]:
x_test = np.dstack((test1.iloc[:,0:64],test2.iloc[:,0:64],test3.iloc[:,0:64],test4.iloc[:,0:64],test5.iloc[:,0:64],test6.iloc[:,0:64],test7.iloc[:,0:64],test8.iloc[:,0:64],test9.iloc[:,0:64]))
x_train = np.dstack((train1.iloc[:,0:64],train2.iloc[:,0:64],train3.iloc[:,0:64],train4.iloc[:,0:64],train5.iloc[:,0:64],train6.iloc[:,0:64],train7.iloc[:,0:64],train8.iloc[:,0:64],train9.iloc[:,0:64]))
print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape([7352,576])
x_test = x_test.reshape([2947,576])


(7352, 64, 9)
(2947, 64, 9)


In [89]:
model = models.Sequential()
model.add(Dense(1024, kernel_initializer="glorot_normal", input_shape=(9*64,)))
model.add(LeakyReLU())
model.add(Dropout(.15))
model.add(BatchNormalization())
# model.add(GaussianNoise(1))
model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dense(6, activation='softmax'))

In [90]:
# Let's use a different optimizer this time
noisy = add_gradient_noise(RMSprop)
model.compile(optimizer="Adamax",
# model.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [91]:
callbacks = [EarlyStopping(monitor='val_loss', patience=16),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [92]:
history = model.fit(x_train, y_train, epochs=250, batch_size=16, validation_data = (x_test, y_test), callbacks=callbacks)
from keras.models import load_model
# model = load_model('best_model.h5', custom_objects={"NoisyRMSprop":noisy()})
model = load_model('best_model.h5')
model.evaluate(x_test, y_test)

Train on 7352 samples, validate on 2947 samples
Epoch 1/250
7352/7352 [==============================] - 6s 850us/step - loss: 0.6940 - acc: 0.7482 - val_loss: 0.5206 - val_acc: 0.8354
Epoch 2/250
7352/7352 [==============================] - 5s 674us/step - loss: 0.3818 - acc: 0.8679 - val_loss: 0.4187 - val_acc: 0.8537
Epoch 3/250
7352/7352 [==============================] - 6s 775us/step - loss: 0.2930 - acc: 0.8943 - val_loss: 0.3534 - val_acc: 0.8778
Epoch 4/250
7352/7352 [==============================] - 6s 773us/step - loss: 0.2501 - acc: 0.9129 - val_loss: 0.3687 - val_acc: 0.8683
Epoch 5/250
7352/7352 [==============================] - 8s 1ms/step - loss: 0.2242 - acc: 0.9185 - val_loss: 0.3156 - val_acc: 0.8755
Epoch 6/250
7352/7352 [==============================] - 9s 1ms/step - loss: 0.1962 - acc: 0.9272 - val_loss: 0.3092 - val_acc: 0.8829
Epoch 7/250
7352/7352 [==============================] - 9s 1ms/step - loss: 0.1945 - acc: 0.9266 - val_loss: 0.3084 - val_acc: 0.8965

[0.23383198726386928, 0.9216152019002375]

### Import data for LSTM

In [1]:
import os
import pandas as pd
import numpy as np
from collections import Counter
os.chdir("UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals")
test1 = pd.read_csv("body_acc_x_test.txt", "\s+", header=None)
test2 = pd.read_csv("body_acc_y_test.txt", "\s+", header=None)
test3 = pd.read_csv("body_acc_z_test.txt", "\s+", header=None)
test4 = pd.read_csv("body_gyro_x_test.txt", "\s+", header=None)
test5 = pd.read_csv("body_gyro_y_test.txt", "\s+", header=None)
test6 = pd.read_csv("body_gyro_z_test.txt", "\s+", header=None)
test7 = pd.read_csv("total_acc_x_test.txt", "\s+", header=None)
test8 = pd.read_csv("total_acc_y_test.txt", "\s+", header=None)
test9 = pd.read_csv("total_acc_z_test.txt", "\s+", header=None)
os.chdir("UCI HAR Dataset/UCI HAR Dataset/train/Inertial Signals")
train1 = pd.read_csv("body_acc_x_train.txt", "\s+", header=None)
train2 = pd.read_csv("body_acc_y_train.txt", "\s+", header=None)
train3 = pd.read_csv("body_acc_z_train.txt", "\s+", header=None)
train4 = pd.read_csv("body_gyro_x_train.txt", "\s+", header=None)
train5 = pd.read_csv("body_gyro_y_train.txt", "\s+", header=None)
train6 = pd.read_csv("body_gyro_z_train.txt", "\s+", header=None)
train7 = pd.read_csv("total_acc_x_train.txt", "\s+", header=None)
train8 = pd.read_csv("total_acc_y_train.txt", "\s+", header=None)
train9 = pd.read_csv("total_acc_z_train.txt", "\s+", header=None)

In [2]:
x_test = np.dstack((test1,test2,test3,test4,test5,test6,test7,test8,test9))
x_train = np.dstack((train1,train2,train3,train4,train5,train6,train7,train8,train9))
os.chdir("UCI HAR Dataset/UCI HAR Dataset/test")
y_test = pd.read_csv("y_test.txt", "\s+", header=None)
os.chdir("UCI HAR Dataset/UCI HAR Dataset/train")
y_train = pd.read_csv("y_train.txt", "\s+", header=None)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(7352, 128, 9)
(2947, 128, 9)
(7352, 1)
(2947, 1)


### One-hot encode y

In [3]:
y_train = pd.get_dummies(y_train[0]).values
y_test = pd.get_dummies(y_test[0]).values
# x_train = x_train.reshape([7352,1152])
# x_test = x_test.reshape([2947,1152])
print(y_test.shape)
y_test

(2947, 6)


array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

### LSTM

In [4]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import pandas as pd
from skimage import io
from PIL import Image
import functools
from matplotlib import pyplot



Using TensorFlow backend.


In [5]:
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import MobileNet
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import BatchNormalization, Flatten, Dense, LeakyReLU, Dropout, GaussianNoise
from tensorflow import set_random_seed
from keras.optimizers import Adam, RMSprop
from keras_gradient_noise import add_gradient_noise
from keras.initializers import glorot_uniform, glorot_normal, he_uniform, he_normal
set_random_seed(1724)


model = models.Sequential()
model.add(LSTM(100, input_shape=(128,9)))
# model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer="glorot_normal", input_shape=(9*128,)))
model.add(LeakyReLU())
model.add(Dropout(.15))
model.add(BatchNormalization())
# model.add(GaussianNoise(1))
model.add(Dense(64))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dense(6, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
# Let's use a different optimizer this time
noisy = add_gradient_noise(Adam)
model.compile(optimizer="adam",
# model.compile(optimizer=noisy(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [7]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
    history = model.fit(x_train, y_train, epochs=250, batch_size=32, validation_data = (x_test, y_test), callbacks=callbacks)
    from keras.models import load_model
    # model = load_model('best_model.h5', custom_objects={"NoisyAdam":noisy()})
    model = load_model('best_model.h5')
    model.evaluate(x_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/250
7352/7352 [==============================] - 59s 8ms/step - loss: 0.9632 - acc: 0.6049 - val_loss: 0.7545 - val_acc: 0.6444
Epoch 2/250
7352/7352 [==============================] - 52s 7ms/step - loss: 0.4287 - acc: 0.8381 - val_loss: 0.5469 - val_acc: 0.8056
Epoch 3/250
7352/7352 [==============================] - 54s 7ms/step - loss: 0.2654 - acc: 0.9048 - val_loss: 0.6630 - val_acc: 0.8422
Epoch 4/250
7352/7352 [==============================] - 48s 6ms/step - loss: 0.2901 - acc: 0.8916 - val_loss: 0.2534 - val_acc: 0.9009
Epoch 5/250
7352/7352 [==============================] - 54s 7ms/step - loss: 0.2194 - acc: 0.9248 - val_loss: 0.2186 - val_acc: 0.9114
Epoch 6/250
7352/7352 [==============================] - 48s 7ms/step - loss: 0.1933 - acc: 0.9286 - val_loss: 0.2367 - val_acc: 0.9108
Epoch 7/250
7352/7352 [==============================] - 47s 6ms/step - loss: 0.1789 - a